In [10]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import dask
import xesmf as xe

from CASutils import calendar_utils

### Read in the coordinates from the SMYLE simulations to set up the reanalysis in the same way

In [24]:
smyle = xr.open_dataset("/glade/campaign/cgd/cas/islas/python_savs/NCAR_CW3E_SMYLE/DATA_SORT/outputzm/Uzm_BSMYLE-CW3E_day_init09.nc")
smyle = smyle.coords

In [25]:
ystart=smyle['init_year'].isel(init_year=0).values
yend=smyle['init_year'].isel(init_year=smyle['init_year'].size-1).values
nyears=yend-ystart+1
initmon=smyle['time'].isel(time=0).time.dt.month.values

In [26]:
jra3q = xr.open_mfdataset("/glade/campaign/cgd/cas/islas/DATASETS/JRA3Q/day/Uzm/*.nc")
jra3q = jra3q['ugrd-pres-an-gauss']
jra3q = jra3q.rename('Uzm')

In [27]:
dpm = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
smon = initmon
emon = (initmon - 1)
if (emon < 1):
    emon = 12    

sdatestr = str(smon).zfill(2)+'-01' ; edatestr = str(emon).zfill(2)+'-'+str(dpm[emon]).zfill(2)

jra3qout=[]

for iyear in np.arange(ystart,yend+1,1):
    print(iyear)
    eyear = iyear+1
    if (emon == 12):
        eyear = iyear

    jra3quse = jra3q.sel(time=slice(str(iyear)+'-'+sdatestr, str(eyear)+'-'+edatestr))

    # drop the 29th Feb on leap years
    jra3quse = jra3quse.sel(time=~((jra3quse.time.dt.month == 2)  & (jra3quse.time.dt.day == 29)))
    jra3quse['time'] = smyle['time'].values
    jra3qout.append(jra3quse)

jra3qout = xr.concat(jra3qout, dim='init_year')
jra3qout = jra3qout.assign_coords(init_year = smyle['init_year'])

1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [28]:
jra3qout.to_netcdf("/glade/campaign/cgd/cas/islas/python/smyle_cw3e/DATA_SORT/outputzm/Uzm_JRA3Q_day_init"+str(initmon).zfill(2)+".nc")